In [1]:
import streamlit as st
import pydeck as pdk
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
# 환경설정
# matplotlib 한글사용을 위한 코드
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
tqdm.pandas()

# 사용자 함수 정의
# 마스터정보의 경도, 위도 정보를  대여소이용정보에 추가
def find_lat_lon(id):
    # 종료_대여소_ID의 경우 'X'인 경우 처리
    if id=='X':
        ret_start = pd.Series(['0','0'])
    else:
        cond_start = (df_master1['대여소_ID']==id)
        ret_start = df_master1.loc[cond_start,['lat','lon']].values[0]

    return pd.Series([ret_start[0], ret_start[1]])

In [3]:
# 데이터 불러오기
# 대여소 마스터파일 : 대여소아이디별 경도, 위도 정보
file_path = './data/서울시 따릉이대여소 마스터 정보.csv'
df_master = pd.read_csv(file_path,encoding='cp949')

# 대여소 이용정보 : 시작대여소, 종료대여소, 이용건수, 이용시간, 이용거리
file_path1_4 = './data/tpss_bcycl_od_statnhm_20240104.csv'
file_path1_5 = './data/tpss_bcycl_od_statnhm_20240105.csv'
file_path1_6 = './data/tpss_bcycl_od_statnhm_20240106.csv'

df_rent1_4 = pd.read_csv(file_path1_4,encoding='cp949')
df_rent1_5 = pd.read_csv(file_path1_5,encoding='cp949')
df_rent1_6 = pd.read_csv(file_path1_6,encoding='cp949')

In [4]:
###########################################################################################
# 데이터 전처리
###########################################################################################

# 경도,위도 컬럼명 변경 : map
df_master.rename(columns={'위도':'lat','경도':'lon'}, inplace = True)

# 이상치 확인(실질적 Nan Data 제거)
# df_master.loc[:,['lat','lon']].plot(kind='box')
# plt.show()

# 경도, 위도가 0인 컬럼이 77개 존재 : 삭제처리
cond = (df_master['lat']==0) | (df_master['lon']==0)
# df_master.loc[cond,:].index
df_master1 = df_master.drop(df_master.loc[cond,:].index,axis=0)
df_master1.reset_index(inplace=True)



# 날짜별 데이터 합치기
# df_rent1 = pd.concat([df_rent1_4,df_rent1_5,df_rent1_6],axis=0)
# 데이터가 많아서 streamlit share에서 에러나서 일단 파일 하나로만 테스트 진행
df_rent1 = df_rent1_4

# 기준날짜 : csv에서 불러오면 dataframe에서는 int64로 들어오기 때문에 datetime으로 변환
df_rent1['기준_날짜'] = df_rent1['기준_날짜'].astype('str')
# df_rent1['기준_날짜'] = pd.to_datetime(df_rent1['기준_날짜'], format='%Y-%m-%d')

# 컬럼명 영문으로 변경
# df_rent1.rename(columns={'기준_날짜':'std_date','집계_기준':'type','기준_시간대':'std_time','시작_대여소_ID':'st_spot_id',
#                          '시작_대여소명':'st_spot_name','종료_대여소_ID':'end_spot_id','종료_대여소명':'end_spot_name',
#                          '전체_건수':'tot_count','전체_이용_분':'tot_use_min','전체_이용_거리':'tot_dist'}, inplace=True)

# 대여소명에 '동'이 표시되어 있지 않은 행 찾기 : 182개 존재
# cond_dong = df_rent1['시작_대여소명'].str.contains('동')
# df_rent1.loc[~cond_dong,:]

# 종료_대여소명 : NaN --> X
df_rent1['종료_대여소명'].fillna('X',inplace=True)



# Feature Engineering
# df_featured_rent : '시작_대여소_동명', '종료_대여소_동명' 추출
df_featured_rent = df_rent1
df_featured_rent['시작_대여소_동명'] = df_rent1['시작_대여소명'].apply(lambda x:x[:x.find('동')+1])
df_featured_rent['종료_대여소_동명'] = df_rent1['종료_대여소명'].apply(lambda x:x[:x.find('동')+1])

# 동별정보를 추가한 df_featured_rent에서 동명이 없는 행을 삭제
cond = ((df_featured_rent['시작_대여소_동명']=='') | (df_featured_rent['종료_대여소_동명']==''))
df_featured_rent.drop(df_featured_rent.loc[cond,:].index, axis=0, inplace=True)


# df_rent1에 경도,위도 정보 추가
df_rent1[['시작_대여소_lat','시작_대여소_lon']] = df_rent1['시작_대여소_ID'].progress_apply(lambda x:find_lat_lon(x))
df_rent1[['종료_대여소_lat','종료_대여소_lon']] = df_rent1['종료_대여소_ID'].progress_apply(lambda x:find_lat_lon(x))


100%|██████████| 179305/179305 [03:24<00:00, 876.63it/s]


In [9]:
save_file = './data/마스터포함_대여소이용정보.csv'
df_rent1.to_csv(save_file,encoding='utf-8',index=False)

In [13]:
df_rent = pd.read_csv(save_file)
df_rent

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리,시작_대여소_동명,종료_대여소_동명,시작_대여소_lat,시작_대여소_lon,종료_대여소_lat,종료_대여소_lon
0,20240104,출발시간,0,ST-1014,신정3동_101_1,ST-676,고척2동_035_1,1,6,2320,신정3동,고척2동,37.508900,126.842682,37.503815,126.851959
1,20240104,출발시간,0,ST-1017,목5동_059_1,ST-310,목4동_047_1,1,6,909,목5동,목4동,37.536503,126.877747,37.532543,126.868729
2,20240104,출발시간,0,ST-1055,강일동_001_1,ST-2744,강일동_001_5,1,93,12860,강일동,강일동,37.568638,127.174805,37.570740,127.172157
3,20240104,출발시간,0,ST-1082,장지동_019_1,ST-2599,장지동_012_3,1,5,910,장지동,장지동,37.480541,127.137016,37.479534,127.126785
4,20240104,출발시간,0,ST-1092,풍납2동_026_1,ST-502,성내1동_007_1,1,5,1063,풍납2동,성내1동,37.532848,127.121048,37.526794,127.130058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179300,20240104,도착시간,2355,ST-914,상계5동_036_1,ST-2711,중계1동_001_1,1,30,1633,상계5동,중계1동,37.660000,127.072968,37.651161,127.076591
179301,20240104,도착시간,2355,ST-935,방배4동_002_1,ST-1916,방배본동_030_1,1,5,1961,방배4동,방배본동,37.488972,126.982719,37.494205,126.989357
179302,20240104,도착시간,2355,ST-942,면목5동_015_1,ST-262,장안2동_053_1,1,19,2777,면목5동,장안2동,37.586372,127.080071,37.572174,127.071388
179303,20240104,도착시간,2355,ST-96,상암동_060_1,ST-3272,성산2동_038_1,1,4,607,상암동,성산2동,37.575665,126.894447,37.574020,126.901024
